In [1]:
# Add the genderComputer library to the path
import os
import sys

path = os.path.join(os.getcwd(), 'vendor', 'genderComputer')
sys.path.insert(0, path)

In [2]:
# Infer the gender of the authors
import analyze_genders

df = analyze_genders.dataframe()
first_author = df['author_position'] == 0

In [3]:
# Group by conference and calculate the ratio of male to female authors
by_conf = df[first_author].drop(columns=['author_position', 'year']).groupby(df['conf']).sum()
by_conf['m/f'] = by_conf['male'] / by_conf['female']

by_conf

,male,female,unisex,unknown,m/f
conf,,,,,
cidr,317,62,14,34,5.112903
edbt,1046,341,46,142,3.067449
icde,2844,936,269,715,3.038462
pvldb,1661,547,97,337,3.036563
sigmod,2763,678,150,520,4.075221
tods,733,134,29,124,5.470149


In [4]:
# Group by year and calculate the ratio of male to female authors
by_year = df[first_author].drop(columns=['author_position', 'year']).groupby(df['year']).sum()
by_year['m/f'] = by_year['male'] / by_year['female']

by_year

,male,female,unisex,unknown,m/f
year,,,,,
1975,18,3,0,6,6.000000
1976,32,4,0,7,8.000000
1977,32,8,0,4,4.000000
1978,38,1,0,10,38.000000
1979,43,4,0,7,10.750000
1980,30,5,1,11,6.000000
1981,40,2,0,13,20.000000
1982,54,6,1,8,9.000000
1983,49,7,0,3,7.000000
